In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.0` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.10.1`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = SparkSession
      .builder()
      .config("spark.sql.shuffle.partitions", 4)
      .master("local[4]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/05/05 10:27:22 INFO SparkContext: Running Spark version 3.2.0
22/05/05 10:27:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/05 10:27:22 INFO ResourceUtils: ==============================================================
22/05/05 10:27:22 INFO ResourceUtils: No custom resources configured for spark.driver.
22/05/05 10:27:22 INFO ResourceUtils: ==============================================================
22/05/05 10:27:22 INFO SparkContext: Submitted application: 551dc5fd-20eb-4080-a0c9-715651cbe8a0
22/05/05 10:27:22 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
22/05/0

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                               


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@b099343
import spark.implicits._



In [2]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [3]:
val lastRace = Window.partitionBy("year")

val lastRaces = spark.read.parquet("../../data/parquet/races.parquet")
    .where(col("year") >= 1990 && col("year") <= 1999)
    .withColumn("round", col("round").cast(IntegerType))
    .withColumn("max", max(col("round")).over(lastRace))
    .where(col("round") === col("max"))
    .select("raceId", "year")

lastRace: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@4253150f
lastRaces: DataFrame = [raceId: string, year: string]

In [4]:
val constructors = Window.partitionBy("constructorId")

val constructorWinners = spark.read.parquet("../../data/parquet/constructor_standings.parquet")
    .join(lastRaces, Seq("raceId"), "right")
    .where(col("position") === 1)
    .select("constructorId", "wins", "year")
    .withColumn("totalChampWins", count(col("constructorId")).over(constructors))
    .withColumn("totalRaceWins", sum(col("wins")).over(constructors).cast(IntegerType))
    .drop("wins")  

constructors: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3338d4f7
constructorWinners: DataFrame = [constructorId: string, year: string ... 2 more fields]

In [5]:
val constructors = spark.read.parquet("../../data/parquet/constructors.parquet")
    .select("constructorId", "name")

constructors: DataFrame = [constructorId: string, name: string]

In [6]:
val mostDominantConstructor = constructorWinners
    .drop("year")
    .dropDuplicates("constructorId")
    .orderBy(col("totalChampWins").desc, col("totalRaceWins").desc)
    .join(constructors, "constructorId")
    .drop("constructorId")

mostDominantConstructor: DataFrame = [totalChampWins: bigint, totalRaceWins: int ... 1 more field]

In [7]:
def run[A](code: => A): Long = {
    val start = System.currentTimeMillis()
    val res = code
    System.currentTimeMillis() - start
}

def average(list: List[Long]):Long = list.sum / list.size.toLong

def timeTest(f: => Long): Long = {
    val list = (1 to 1).map(_ => f).toList
    average(list)
}

val res = timeTest(run(mostDominantConstructor.collect))

defined function run
defined function average
defined function timeTest
res: Long = 1637L